In [ ]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt




# Proposal distribution: 1/Z * f_n
p_n = st.norm(0, 1)

x = np.arange(-10, 5, 0.1)

n_inter = 50  # num of intermediate dists
betas = np.linspace(0, 1, n_inter)

# Sampling
n_samples = 100
samples = np.zeros(n_samples)
weights = np.zeros(n_samples)

for t in range(n_samples):
    # Sample initial point from q(x)
    x = p_n.rvs()
    w = 1

    for n in range(1, len(betas)):
        # Transition
        x = T(x, lambda x: f_j(x, betas[n]), n_steps=5)

        # Compute weight in log space (log-sum):
        # w *= f_{n-1}(x_{n-1}) / f_n(x_{n-1})
        w += np.log(f_j(x, betas[n])) - np.log(f_j(x, betas[n-1]))

    samples[t] = x
    weights[t] = np.exp(w)  # Transform back using exp


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
from torch.distributions.one_hot_categorical import OneHotCategorical



In [14]:
sample = distrib.sample((20,))

In [27]:
def f_n(x):
    return F.linear(sample, W)

In [63]:
p_0 = F.softmax(torch.randn(31,21),1).view(-1)
W = torch.rand((31*21,31*21))
distrib = OneHotCategorical(p_0.view(31,21))
log_p_0 = torch.log(p_0)

def log_f0(x):
    return (log_p_0*x).sum(-1)

def log_fn(x):
    return F.linear(x, W).sum(-1)

def log_fj(x, beta):
    return log_f0(x) * beta + log_fn(x)*(1-beta)

def T(x, f, n_steps=10):
    for t in range(n_steps):
        x_prime = x + torch.randn_like(x)
        a = f(x_prime) - f(x)
        idx = torch.where(torch.log(torch.rand_like(a)) < a)
        x[idx] = x_prime[idx]
    return x

n_inter = 2000  # num of intermediate dists
betas = torch.linspace(0, 1, n_inter)

# Sampling
n_samples = 100
samples = distrib.sample((n_samples,)).view(n_samples, -1)
weights = torch.zeros(n_samples)

last_beta = 0.
for beta in betas[1:]:
    # Transition
    samples = T(samples, lambda x: log_fj(x, beta), n_steps=5)

    # Compute weight in log space (log-sum):
    weights += log_fj(samples, beta) - log_fj(samples, last_beta)
    last_beta = beta

In [56]:
F.sum(log_p_0*x)

AttributeError: module 'torch.nn.functional' has no attribute 'sum'

In [75]:
class AIS(object):
    def __init__(self, log_distrib_0, log_distrib_n, n_inter = 2000, n_samples = 20):
        self.

tensor([-16946082., -17002200., -16474005., -16618410., -16947744., -17127642.,
        -16503876., -16539582., -16958064., -16785836., -17095802., -16795840.,
        -16499058., -16773774., -16525460., -16561337., -16678338., -16709349.,
        -16737122., -16643770., -16933270., -17033604., -16943240., -17071156.,
        -16446879., -17065806., -16755993., -16871228., -17059512., -16767165.,
        -16990088., -16652168., -16428339., -16884848., -16870760., -16867574.,
        -16582169., -16995146., -16849628., -16790010., -16883712., -16201873.,
        -16762177., -16676338., -16392031., -16616135., -16577959., -16436741.,
        -16182006., -16888576., -16525746., -16415147., -17437320., -16670569.,
        -16021310., -17118796., -16555226., -16632096., -16239332., -16252348.,
        -16528051., -16620051., -16260128., -16425049., -17111796., -16408548.,
        -17194714., -16988550., -16396600., -16678232., -16593203., -16318747.,
        -16673906., -16979738., -1676264